<a href="https://colab.research.google.com/github/squeeko/DeepChem_projects/blob/master/DC_1_BasicTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 1: The Basic Tools of the Deep Life Sciences

Welcome to DeepChem's introductory tutorial for the deep life sciences. This series of notebooks is a step-by-step guide for you to get to know the new tools and techniques needed to do deep learning for the life sciences. We'll start from the basics, assuming that you're new to machine learning and the life sciences, and build up a repertoire of tools and techniques that you can use to do meaningful work in the life sciences.

Scope: This tutorial will encompass both the machine learning and data handling needed to build systems for the deep life sciences.


## Setup
The first step is to get DeepChem up and running. We recommend using Google Colab to work through this tutorial series. You'll need to run the following commands to get DeepChem installed on your colab notebook. Note that this will take something like 5 minutes to run on your colab instance.

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py

import conda_installer
conda_installer.install()
!root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0   9483      0 --:--:-- --:--:-- --:--:--  9483


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


/bin/bash: root/miniconda/bin/conda: No such file or directory


In [2]:
!pip install --pre deepchem

     |████████████████████████████████| 399kB 2.7MB/s 
  Created wheel for deepchem: filename=deepchem-2.4.0rc1.dev20200921220318-cp36-none-any.whl size=495597 sha256=dade34ec8ed237af22a4bc2a49a3cc749f89b63f00e79bfbc5a80758f4816c6b
  Stored in directory: /root/.cache/pip/wheels/a4/a6/b4/ba3ded5d5350c268f459a51638e265939b9ab601f1ce3e9732
Successfully built deepchem


In [3]:
import deepchem as dc
print(dc.__version__)

2.4.0-rc1.dev


# Training a Model with DeepChem: A First Example

Deep learning can be used to solve many sorts of problems, but the basic workflow is usually the same. Here are the typical steps you follow.


1.   Select the data set you will train your model on (or create a new data set if there isn't an existing suitable one).

2.   Create the model.

3.   Train the model on the data.

4.   Evaluate the model on an independent test set to see how well it works.

5.   Use the model to make predictions about new data.










With DeepChem, each of these steps can be as little as one or two lines of Python code. In this tutorial we will walk through a basic example showing the complete workflow to solve a real world scientific problem.

The problem we will solve is predicting the solubility of small molecules given their chemical formulas. This is a very important property in drug development: if a proposed drug isn't soluble enough, you probably won't be able to get enough into the patient's bloodstream to have a therapeutic effect. The first thing we need is a data set of measured solubilities for real molecules. One of the core components of DeepChem is MoleculeNet, a diverse collection of chemical and molecular data sets. For this tutorial, we can use the Delaney solubility data set.

In [4]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

I won't say too much about this code right now. We will see many similar examples in later tutorials. There are two details I do want to draw your attention to. First, notice the featurizer argument passed to the load_delaney() function. Molecules can be represented in many ways. We therefore tell it which representation we want to use, or in more technical language, how to "featurize" the data. Second, notice that we actually get three different data sets: a training set, a validation set, and a test set. Each of these serves a different function in the standard deep learning workflow.

Now that we have our data, the next step is to create a model. We will use a particular kind of model called a "graph convolutional network", or "graphconv" for short.

In [5]:
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)

Here again I will not say much about the code. Later tutorials will give lots more information about GraphConvModel, as well as other types of models provided by DeepChem.

We now need to train the model on the data set. We simply give it the data set and tell it how many epochs of training to perform (that is, how many complete passes through the data to make).

In [6]:
model.fit(train_dataset, nb_epoch=100)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.11356877326965333

If everything has gone well, we should now have a fully trained model! But do we? To find out, we must evaluate the model on the test set. We do that by selecting an evaluation metric and calling evaluate() on the model. For this example, let's use the Pearson correlation, also known as r2, as our metric. We can evaluate it on both the training set and test set.

In [8]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score: ", model.evaluate(train_dataset, [metric], transformers))
print("Test set score: ", model.evaluate(test_dataset, [metric], transformers))

Training set score:  {'pearson_r2_score': 0.9126107726551816}
Test set score:  {'pearson_r2_score': 0.7683041871585818}


Notice that it has a higher score on the training set than the test set. Models usually perform better on the particular data they were trained on than they do on similar but independent data. This is called "overfitting", and it is the reason it is essential to evaluate your model on an independent test set.

Our model still has quite respectable performance on the test set. For comparison, a model that produced totally random outputs would have a correlation of 0, while one that made perfect predictions would have a correlation of 1. Our model does quite well, so now we can use it to make predictions about other molecules we care about.

Since this is just a tutorial and we don't have any other molecules we specifically want to predict, let's just use the first ten molecules from the test set. For each one we print out the chemical structure (represented as a SMILES string) and the predicted solubility.

In [9]:
solubilities = model.predict_on_batch(test_dataset.X[:10])
for molecule, solubility in zip(test_dataset.ids, solubilities):
  print(solubility, molecule)

[-1.9822992] C1c2ccccc2c3ccc4ccccc4c13
[0.38088506] COc1ccccc1Cl
[-1.1326052] COP(=S)(OC)Oc1cc(Cl)c(Br)cc1Cl
[0.15392745] ClC(Cl)CC(=O)NC2=C(Cl)C(=O)c1ccccc1C2=O
[-1.3780257] ClC(Cl)C(c1ccc(Cl)cc1)c2ccc(Cl)cc2 
[2.5349941] COC(=O)C=C
[0.16927208] CN(C)C(=O)Nc2ccc(Oc1ccc(Cl)cc1)cc2
[1.0321488] N(=Nc1ccccc1)c2ccccc2
[0.17459251] CC(C)c1ccc(C)cc1
[1.4632709] Oc1c(Cl)cccc1Cl
